In [23]:
import newspaper

_20m_paper = newspaper.build('https://www.20minutes.fr/politique/', memoize_articles=False,fetch_images=False)

papers = []
urls_set = set()
for article in _20m_paper.articles:
   # check to see if the article url is not within the urls_set
   if article.url not in urls_set:
     # add the unique article url to the set
     urls_set.add(article.url)
     # remove all links for article commentaires
     if not str(article.url).endswith('commentaires'):
        papers.append(article.url)

print(len(papers)) 

176


In [24]:
import newspaper
import requests
from newspaper.utils import BeautifulSoup

papers = []
urls_set = set()

base_url = 'https://www.20minutes.fr/'

_20m_paper = newspaper.build(base_url, 
fetch_images=False, memoize_articles=False)

for article in _20m_paper.articles:
   # check to see if the article url is not within the urls_set
   if article.url not in urls_set:
     # add the unique article url to the set
     urls_set.add(article.url)
     # remove all links for article commentaires
     if not str(article.url).endswith('commentaires'):
        papers.append(article.url)

 
_20_urls = {'societe': base_url+'societe',
             'politique': base_url+'politique',
             'faits_divers': base_url+'faits_divers',
             'monde': base_url+'monde',
             'culture':base_url+'culture',
             'people': base_url+'people',
             'sports': base_url+'sports',
             'economie': base_url+'economie',
             'sciences': base_url+'sciences',
             'planete': base_url+'planete',
             'high-tech' : base_url+'high-tech',
             }


for category, url in _20_urls.items():
   raw_html = requests.get(url)
   soup = BeautifulSoup(raw_html.text, 'html.parser')
   for articles_tags in soup.findAll('div', {'class': 'articles'}):
      for article_href in articles_tags.find_all('a', href=True):
         if not str(article_href['href']).endswith('commentaires'):
           urls_set.add(article_href['href'])
           papers.append(article_href['href'])

print(len(papers))

176


In [181]:
import time
import requests
from bs4 import BeautifulSoup
import json
import string
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


base_url = 'https://www.20minutes.fr/'

papers = []
urls_set = set()

browser = webdriver.Chrome('/mnt/c/users/tdenimal/Downloads/chromedriver_win32/chromedriver.exe')
wait = WebDriverWait(browser, 10)


browser.get(base_url+'politique')
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")


while True:
    try:
        time.sleep(1)
        show_more = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@type="button"][contains(.,"Voir plus")]')))
        browser.execute_script("arguments[0].click();", show_more)
    except Exception as e:
            print(e)
            break
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    for articles_tags in soup.findAll('article'):
        for article_href in articles_tags.find_all('a', href=True):
            if not str(article_href['href']).endswith('commentaires'):
                urls_set.add(article_href['href'])
                papers.append(article_href['href'])

    print(len(papers))

30
70
120
180
250
330
420
520
630
750
880
1020
1170
1330
1500
1680
1870
2070
2280
2500
2730
2970
3220
3480
3750
4030
4320
4620
4930
5250
5580
5920
6270
6630
7000
7380
7770
8170
8580
9000
9430
9870
10320
10780
11250
11730
12220
12720
13230
13750
14280
14820
15370
15930
16500
17080
17670
18270
18880
19500
20130
20770


KeyboardInterrupt: 

In [27]:
import spacy
from spacy import displacy
from collections import Counter
import fr_core_news_lg
nlp = fr_core_news_lg.load()

In [208]:
import numpy as np
from newspaper import Article

#ners = []
for i,p in enumerate(papers[17462:]):
    try:
        article = Article(url=base_url+p[1:], language='fr')
        article.download()
        article.parse()
    except:
        print("Retry article #"+str(i))
        pass

    doc = nlp(article.text)
    ner = [(X.text, X.label_) for X in doc.ents]
#Return only unique tupe
    ner = [tuple(i) for i in np.unique(ner, axis=0)]
    ners += [ner]

In [94]:
import numpy as np
import itertools
from scipy.sparse import csr_matrix


def create_co_occurences_matrix(allowed_words, documents):
    print(f"allowed_words:\n{allowed_words}")
    print(f"documents:\n{documents}")
    word_to_id = dict(zip(allowed_words, range(len(allowed_words))))
    #List of ids in each document
    documents_as_ids = [np.sort([word_to_id[w] for w in doc if w in word_to_id]).astype('uint32') for doc in documents]
    
    row_ind, col_ind = zip(*itertools.chain(*[[(i, w) for w in doc] for i, doc in enumerate(documents_as_ids)]))
    data = np.ones(len(row_ind), dtype='uint32')  # use unsigned int for better memory utilization
    max_word_id = max(itertools.chain(*documents_as_ids)) + 1
    docs_words_matrix = csr_matrix((data, (row_ind, col_ind)), shape=(len(documents_as_ids), max_word_id))  # efficient arithmetic operations with CSR * CSR
    words_cooc_matrix = docs_words_matrix.T * docs_words_matrix  # multiplying docs_words_matrix with its transpose matrix would generate the co-occurences matrix
    words_cooc_matrix.setdiag(0)
    print(f"words_cooc_matrix:\n{words_cooc_matrix.todense()}")
    return words_cooc_matrix, word_to_id 

In [211]:
unique_values = [tuple(i) for i in np.unique(np.concatenate(ners), axis=0)]


In [215]:
words_cooc_matrix, word_to_id = create_co_occurences_matrix(unique_values, ners)

R'), ('Edouard Philippe', 'PER'), ('François Hollande', 'PER'), ('Jacques Chirac', 'PER'), ('Matignon', 'PER'), ('Olivier Coret', 'PER'), ('Premier ministre', 'PER'), ('Républicains', 'ORG'), ('SIPA', 'ORG'), ('Ve République', 'ORG'), ('du Havre', 'LOC'), ('plateau de France', 'LOC')], [('AFP', 'ORG'), ('Christophe Simon', 'PER'), ('EELV', 'ORG'), ('Elysée', 'ORG'), ('Emmanuel Macron', 'PER'), ('Insoumis', 'ORG'), ('Jean-Luc Mélenchon', 'PER'), ('LREM', 'ORG'), ('La République en Marche', 'MISC'), ('Marseille', 'LOC'), ('président de la République', 'PER')], [('20 Minutes', 'MISC'), ('20 Minutes\n\nFrédérique Thiennot', 'MISC'), ('André Trigano', 'PER'), ('André Trigano\u200b', 'PER'), ('Appaméens', 'LOC'), ('Ariège', 'LOC'), ('B. Colin', 'PER'), ('Daniel Mémain', 'PER'), ('DvD', 'MISC'), ('Frédérique Thiennot', 'PER'), ('Pamiers', 'LOC')], [('Absolument', 'PER'), ('Allez', 'MISC'), ('Bedarrides', 'LOC'), ('Brune Poirson', 'PER'), ('CNews', 'ORG'), ('Carpentras', 'LOC'), ('Chenu', 'PER

In [217]:
#Create inv id-> word
id_to_word = {}
for k, v in word_to_id.items():
    id_to_word[v] = k

In [177]:
import logging
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable

class App:

    def __init__(self, uri):
        self.driver = GraphDatabase.driver(uri)

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_entity(self, entity_name,entity_type):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            session.write_transaction(self._create_entity,entity_name,entity_type)

    def create_relationship(self, entity1_name,
                            entity1_type,
                            entity2_name,
                            entity2_type,
                            weight):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            session.write_transaction(self._create_relationship,entity1_name,entity1_type,
                                     entity2_name,entity2_type,weight)

    @staticmethod
    def _create_entity(tx, entity_name,entity_type):

        if entity_type == "PER":
            query = (
                "CREATE (p1:Person { name: $entity_name }) "
                "RETURN p1"
            )
        elif entity_type == "ORG":
            query = (
                "CREATE (p1:Org { name: $entity_name }) "
                "RETURN p1"
            )
        elif entity_type == "LOC":
            query = (
                "CREATE (p1:Location { name: $entity_name }) "
                "RETURN p1"
            )
        else:
            query = (
                "CREATE (p1:Misc { name: $entity_name }) "
                "RETURN p1"
            )

        result = tx.run(query, entity_name=entity_name,entity_type=entity_type)
        try:
            return [{"p1": record["p1"]["name"]}
                    for record in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise


    @staticmethod
    def _create_relationship(tx, entity1_name,entity1_type, entity2_name,entity2_type,weight):

        # To learn more about the Cypher syntax,
        # see https://neo4j.com/docs/cypher-manual/current/

        # The Reference Card is also a good resource for keywords,
        # see https://neo4j.com/docs/cypher-refcard/current/

        if entity1_type == "PER":
            ent1 = "MATCH (p1: Person { name: $entity1_name })"
        elif entity1_type == "ORG":
            ent1 = "MATCH (p1: Org { name: $entity1_name })"
        elif entity1_type == "LOC":
            ent1 = "MATCH (p1: Location { name: $entity1_name })"
        else:
            ent1 = "MATCH (p1: Misc { name: $entity1_name })"

        if entity2_type == "PER":
            ent2 = "MATCH (p2: Person { name: $entity2_name })"
        elif entity2_type == "ORG":
            ent2 = "MATCH (p2: Org { name: $entity2_name })"
        elif entity2_type == "LOC":
            ent2 = "MATCH (p2: Location { name: $entity2_name })"
        else:
            ent2 = "MATCH (p2: Misc { name: $entity2_name })"

        query = (
            ent1 + ent2 +
            "CREATE (p1)-[:co{ weight: $weight }]->(p2) "
            "RETURN p1, p2"
        )
        result = tx.run(query, 
                        entity1_name=entity1_name, 
                        entity2_name=entity2_name,
                        entity1_type=entity1_type, 
                        entity2_type=entity2_type,
                        weight=weight)
        try:
            return [{"p1": record["p1"]["name"], "p2": record["p2"]["name"]}
                    for record in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise


# if __name__ == "__main__":
#     # See https://neo4j.com/developer/aura-connect-driver/ for Aura specific connection URL.
#     scheme = "bolt"  # Connecting to Aura, use the "neo4j+s" URI scheme
#     host_name = "localhost"
#     port = 7687
#     url = "{scheme}://{host_name}:{port}".format(scheme=scheme, host_name=host_name, port=port)
#     app = App(url)
#     app.create_friendship("Alice", "David")
#     app.close()

In [180]:
#Create all entities

scheme = "neo4j"  # Connecting to Aura, use the "neo4j+s" URI scheme
#     
host_name = "localhost"
port = 11003

url = "{scheme}://{host_name}:{port}".format(scheme=scheme, host_name=host_name, port=port)
app = App(url)

#Create entities
for k in word_to_id.keys():
    app.create_entity(k[0],k[1])

#Create relationships
for l in range(0,words_cooc_matrix.shape[0]):
    for c in range(0, words_cooc_matrix.shape[1]):
        if words_cooc_matrix[l,c] > 0:
            app.create_relationship(id_to_word[l][0],id_to_word[l][1],
                                    id_to_word[c][0],id_to_word[c][1],int(words_cooc_matrix[l,c]))

app.close()